# Data Preprocessing

## Helper class to create the one-hot vectors for our dataset.

In [50]:
SOS_token = 0 #start of sentence
EOS_token = 1 #end of sentence

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            

## Preprocess files

Transform unicode in ascii and convert to lower case.

In [14]:
import re, unicodedata

def unicodeToAscii(s):
    return ''.join( c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

## Read file

Split line into pairs (english / other language)

In [25]:
def readLangs(lang1, lang2, reverse = False):
    print("Reading lines...")
    lines = open ('data/%s-%s/%s.txt'%(lang1, lang2, lang1), encoding = 'utf-8').read().strip().split('\n')
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    else:
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    return input_lang, output_lang, pairs

## Make it easier

Keep only sort sentences that translate to the form "I am", "He is", etc.

In [22]:
MAX_LENGTH = 10

eng_prefixes = ('i am ', 'i m ','you are ', 'you re ', 'he is ', ' he s ', 'she is ', 'she s ', 'we are ', 'we re ', 'they are ', 'they re ')

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

## Full preprocess

In [52]:
import random

def prepareData(lang1, lang2, reverse = False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print('Read %d sentence pairs'%len(pairs))
    pairs = filterPairs(pairs)
    print('Trimmed to %d sentence pairs'%len(pairs))
    print('Counting words...')
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print('Counted words:')
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('fra', 'eng', True)
example = random.choice(pairs)
print(example)

Reading lines...
Read 149861 sentence pairs
Trimmed to 10535 sentence pairs
Counting words...
Counted words:
fra 4279
eng 2737
['nous sommes impuissants .', 'we re helpless .']


In [69]:
example = ['nous sommes fatigues .', 'we re helpless .']

# Seq2seq model

## Encoder

In [103]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

USE_CUDA = False
INPUT_SIZE = input_lang.n_words
HIDDEN_SIZE = 256
OUTPUT_SIZE = output_lang.n_words

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN,self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1,1,-1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden
    
    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if USE_CUDA:
            return result.cuda()
        else:
            return result
        
encoder = EncoderRNN(INPUT_SIZE,HIDDEN_SIZE)

In [106]:
encoder_hidden = encoder.initHidden()
input_length = input_var.size()[0]
encoder_outputs = Variable(torch.zeros(MAX_LENGTH, encoder.hidden_size))
encoder_outputs = encoder_outputs.cuda() if USE_CUDA else encoder_outputs
for ei in range(input_length):
    encoder_output, encoder_hidden = encoder(input_var[ei], encoder_hidden)
    encoder_outputs[ei] = encoder_output[0][0]     
print(encoder_outputs)

Variable containing:
-0.2406 -0.3146 -0.1436  ...  -0.2661 -0.0426  0.2582
-0.4276 -0.5299  0.2215  ...   0.2187  0.1916  0.5870
-0.3860 -0.1503  0.0640  ...   0.0087 -0.0737  0.1661
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
[torch.FloatTensor of size 10x256]



## Decoder

In [92]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN,self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.embedding = nn.Embedding(output_size,hidden_size)
        self.gru = nn.GRU(hidden_size,hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self,input,hidden):
        output = self.embedding(input).view(1,1,-1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]),dim=1)
        return output, hidden
    
    def initHidden(self):
        return Variable(torch.zeros(1,1,self.hidden_size))
        if USE_CUDA:
            return result.cuda()
        else:
            return result
    

## Decoder with Attention

In [132]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p = 0.1, max_length = MAX_LENGTH):
        super(AttnDecoderRNN,self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attn = nn.Linear(hidden_size *2, max_length)
        self.attn_combine = nn.Linear(hidden_size*2, hidden_size)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size,output_size)
        
    def forward(self,input,prev_hidden,encoder_outputs):
        embedded = self.embedding(input).view(1,1,-1)
        embedded = self.dropout(embedded) #1, 1, 256
        # prev_hidden: 1, 1, 256
        attn_weights = self.attn(torch.cat([embedded,prev_hidden],2)) #1, 1, 10
        attn_weights = F.softmax(attn_weights,dim=2) #1, 1, 10
        encoder_outputs = encoder_outputs.unsqueeze(0) #1, 10, 256
        attn_applied = torch.bmm(attn_weights,encoder_outputs) # 1, 1, 256
        output = self.attn_combine(torch.cat([embedded,attn_applied],2)) #1, 1, 256
        output = F.relu(output) #1, 1, 256
        output, hidden = self.gru(output,prev_hidden) #1, 1, 256 ; 1, 1, 256
        output = self.out(output)
        output = F.softmax(output,dim=1)
        return output, hidden, attn_weights #1, 1, 256 ; 1, 1, 256 ; 1, 1, 10
    
    def initHidden(self):
        result = Variable(torch.zeros(1,1,self.hidden_size))
        if USE_CUDA:
            return result.cuda()
        else:
            return result
        
decoder = AttnDecoderRNN(HIDDEN_SIZE,OUTPUT_SIZE)
decoder_hidden = encoder_hidden
target_length = target_var.size()[0]
loss = 0
decoder_input = Variable(torch.LongTensor([[SOS_token]]))
decoder_input = decoder_input.cuda() if USE_CUDA else decoder_input
criterion = nn.NLLLoss()

# Feed the target as the next input
for di in range(target_length):
    decoder_output, decoder_hidden, decoder_attn = decoder(decoder_input, decoder_hidden, encoder_outputs)
    loss += criterion(decoder_output[0], target_var[di])
    decoder_input = target_var[di]
print(loss)

Variable containing:
-5
[torch.FloatTensor of size 1]



# Training

## Prepare data

Get one hot vectors for each word in sentence.

In [70]:
def indexesFromSentence(lang,sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def variableFromSentence(lang,sentence):
    indexes = indexesFromSentence(lang,sentence)
    indexes.append(EOS_token)
    result = Variable(torch.LongTensor(indexes).view(-1,1))
    if USE_CUDA:
        return result.cuda()
    else:
        return result
    
def variablesFromPair(pair):
    input_variable = variableFromSentence(input_lang,pair[0])
    target_variable = variableFromSentence(output_lang,pair[1])
    return (input_variable, target_variable)

input_var, target_var = variablesFromPair(example)

## Training with teacher forcing

In [140]:
TEACHER_FORCING_RATIO = 0.5
import torch.optim as optim

def train(input_var, target_var, encoder, decoder, encoder_optimizer, decoder_optimizer, 
          criterion, max_length=MAX_LENGTH):
    
    encoder_hidden = encoder.initHidden()
    decoder_hidden = encoder_hidden
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_var.size()[0]
    target_length = target_var.size()[0]
    
    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if USE_CUDA else encoder_outputs
    
    loss = 0
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_var[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0][0]
        
    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda() if USE_CUDA else decoder_input
    
    use_teacher_forcing = True if random.random() < TEACHER_FORCING_RATIO else False
    
    if use_teacher_forcing:
        # Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attn = decoder(decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output[0], target_var[di])
            decoder_input = target_var[di]
            
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attn = decoder(decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output[0], target_var[di])
            topv, topi = decoder_output[0].data.topk(1)
            ni = topi[0][0]
            decoder_input = Variable(torch.LongTensor([[ni]]))
            decoder_input = decoder_input.cuda() if USE_CUDA else decoder_input
            if ni == EOS_token:
                break
                
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0] / target_length

encoder_opt = optim.SGD(encoder.parameters(), lr=0.01)
decoder_opt = optim.SGD(encoder.parameters(), lr=0.01)
train(input_var,target_var,encoder,decoder,encoder_opt,decoder_opt,nn.NLLLoss())


-1.0

## Keep track of time

In [142]:
import time, math

def asMinutes(s):
    m = math.floor(s/60)
    s -= m*60
    return '%dm %ds' % (m,s)

def timeSince(since,percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s),asMinutes(rs))

## Plot results

In [135]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy

def showPlot(points):
    plt.figure()
    fid, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

## Full process

In [ ]:
def trainIters(encoder,decoder,n_iters,learning_rate=0.01,print_every=1000,plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0 # reset every print_every
    plot_loss_total = 0 # reset every plot_every
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr = learning_rate)
    decoder_optimizer = optim.SGD(encoder.parameters(), lr = learning_rate)
    training_pairs = [variablesFromPair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()
    
    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_variable = training_pair[0]
        target_variable = training_pair[1]
        
        loss = train(input_variable,target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),iter, iter / n_iters *100, print_loss_avg))
            
        if iter % plot_every == 0:
            plot_loss_avg = print_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_avg = 0
            
    showPlot(plot_losses)
    
trainIters(encoder,decoder,10000,500)

1m 37s (- 14m 40s) (1000 10%) -1.0000
3m 15s (- 13m 0s) (2000 20%) -1.0000
